In [25]:
!pip install yfinance statsmodels --quiet



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\lenovo ideapad\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
plt.style.use("seaborn-v0_8")
from sklearn.preprocessing import StandardScaler


In [27]:
nifty100 = [
"ADANIENT.NS","ADANIPORTS.NS","ASIANPAINT.NS","AXISBANK.NS","BAJAJ-AUTO.NS",
"BAJFINANCE.NS","BAJAJFINSV.NS","BPCL.NS","BHARTIARTL.NS","BOSCHLTD.NS",
"BRITANNIA.NS","CIPLA.NS","COALINDIA.NS","DIVISLAB.NS","DRREDDY.NS",
"EICHERMOT.NS","GRASIM.NS","HCLTECH.NS","HDFC.NS","HDFCBANK.NS",
"HDFCLIFE.NS","HEROMOTOCO.NS","HINDALCO.NS","HINDUNILVR.NS","ICICIBANK.NS",
"ITC.NS","INDUSINDBK.NS","INFY.NS","JSWSTEEL.NS","KOTAKBANK.NS",
"LT.NS","M&M.NS","MARUTI.NS","NESTLEIND.NS","NTPC.NS",
"OIL.NS","ONGC.NS","POWERGRID.NS","RELIANCE.NS","SHREECEM.NS",
"SBIN.NS","SUNPHARMA.NS","TCS.NS","TATACONSUM.NS","TATAMOTORS.NS",
"TATASTEEL.NS","TECHM.NS","TITAN.NS","ULTRACEMCO.NS","UPL.NS",
"WIPRO.NS","COFORGE.NS","BANDHANBNK.NS","MOTHERSUMI.NS","SBILIFE.NS",
"PEL.NS","NTPC.NS","HINDPETRO.NS","VINATIORGA.NS","ADANIGREEN.NS",
"AUROPHARMA.NS","BHEL.NS","CADILAHC.NS","CANBK.NS","CHOLAFIN.NS",
"DLF.NS","GAIL.NS","HAVELLS.NS","IBULHSGFIN.NS","IDEA.NS",
"INDIGO.NS","LTIM.NS","MFSL.NS","MPI.NS","NAUKRI.NS",
"NMDC.NS","PAGEIND.NS","PEL.NS","PNB.NS","RECLTD.NS",
"SUNTV.NS","TATAPOWER.NS","TATACHEM.NS","TVSMOTOR.NS","WELCORP.NS",
"ZOMATO.NS","BALKRISIND.NS","BERGEPAINT.NS","BIOCON.NS","CROMPTON.NS",
"EXIDEIND.NS","GLENMARK.NS","HDFCLIFE.NS","IDFCFIRSTB.NS","INDHOTEL.NS",
"JUBLFOOD.NS","LTI.NS","MCDOWELL-N.NS","NIACL.NS","PNBHOUSING.NS"
]
# Remove duplicates & ensure unique
nifty100 = list(dict.fromkeys(nifty100))
print("Tickers loaded:", len(nifty100))


Tickers loaded: 97


In [28]:
# ===============================
# 1️⃣ Download prices 2019–2025
# ===============================
start_date = "2019-01-01"
end_date = "2025-12-31"   # extended to allow proper OOS + rolling warmup

print("Downloading price data ...")
raw = yf.download(nifty100, start=start_date, end=end_date,
                  group_by="ticker", auto_adjust=True, threads=True)

# Extract Close prices robustly
if ("Close" in raw.columns) or (
    isinstance(raw.columns, pd.MultiIndex) and "Close" in raw.columns.levels[1]
):
    prices = pd.DataFrame()
    for t in nifty100:
        try:
            prices[t] = raw[t]["Close"]
        except Exception:
            pass
else:
    prices = raw.copy()

# Clean data
prices.dropna(axis=1, how='all', inplace=True)
prices.dropna(axis=0, how='any', inplace=True)
prices = prices.loc[:, prices.std() > 0]

print("Final stocks with complete data:", prices.shape[1])
prices.tail()

# ===============================
# 2️⃣ Define selection vs backtest windows
# ===============================
selection_prices = prices.loc["2019-01-01":"2024-12-31"]   # for pair selection
backtest_prices  = prices.loc["2025-01-01":"2025-12-31"]   # OOS backtest


[*********************100%***********************]  97 of 97 completed

10 Failed downloads:
['CADILAHC.NS', 'HDFC.NS', 'MCDOWELL-N.NS', 'ZOMATO.NS', 'MPI.NS', 'IBULHSGFIN.NS', 'LTI.NS', 'PEL.NS', 'MOTHERSUMI.NS']: YFTzMissingError('possibly delisted; no timezone found')
['TATAMOTORS.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-12-31) (Yahoo error = "No data found, symbol may be delisted")')


Final stocks with complete data: 87


In [29]:
# # ===============================
# # 3️⃣ Candidate pairs selection (2019–2024)
# # ===============================

# # Use only selection window
# returns = selection_prices.pct_change().dropna()
# corr = returns.corr()

# corr_threshold = 0.55       # tuneable
# max_candidate_pairs = 400   # safety cap

# candidate_pairs = []
# cols = selection_prices.columns.tolist()
# for i in range(len(cols)):
#     for j in range(i+1, len(cols)):
#         c = corr.iloc[i, j]
#         if c >= corr_threshold:
#             candidate_pairs.append((cols[i], cols[j], c))

# # Keep top by correlation
# candidate_pairs = sorted(candidate_pairs, key=lambda x: x[2], reverse=True)
# if len(candidate_pairs) > max_candidate_pairs:
#     candidate_pairs = candidate_pairs[:max_candidate_pairs]

# print("Candidate pairs after correlation filter (2019-2024):", len(candidate_pairs))


In [30]:
from sklearn.cluster import AgglomerativeClustering
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Use only selection window
returns = selection_prices.pct_change().dropna()

# Correlation distance
corr_matrix = returns.corr()
dist_matrix = np.sqrt(2 * (1 - corr_matrix))
tickers = corr_matrix.columns.tolist()

# Agglomerative clustering
n_clusters = 20
clustering = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric="precomputed",
    linkage="average"
)
labels = clustering.fit_predict(dist_matrix.values)

# Generate candidate pairs
selected = []

def is_cointegrated(y, x, pval_threshold=0.05):
    beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
    spread = y - beta * x
    pval = adfuller(spread)[1]
    return pval < pval_threshold

for cluster_id in np.unique(labels):
    idx = np.where(labels == cluster_id)[0]
    if len(idx) < 2:
        continue
    for i in range(len(idx)):
        for j in range(i + 1, len(idx)):
            stock_i = tickers[idx[i]]
            stock_j = tickers[idx[j]]
            # Check cointegration in-sample
            df_pair = selection_prices[[stock_i, stock_j]].dropna()
            if len(df_pair) < 60:
                continue
            if not is_cointegrated(df_pair[stock_i], df_pair[stock_j]):
                continue
            similarity = corr_matrix.iloc[idx[i], idx[j]]
            selected.append((stock_i, stock_j, similarity))

# Safety cap
max_candidate_pairs = 400
selected = sorted(selected, key=lambda x: abs(x[2]), reverse=True)[:max_candidate_pairs]
selected = pd.DataFrame(selected, columns=["A", "B", "corr"])
print("Candidate pairs after clustering + cointegration:", len(selected))


C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

Candidate pairs after clustering + cointegration: 172


C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_30244\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

In [31]:
# # ===============================
# # 4️⃣ Cointegration / half-life / beta (2019–2024 selection)
# # ===============================

# cointegrated = []

# # Looser but practical filters
# ADF_P_THRESHOLD = 0.08       # allows some pairs that are reasonably stationary
# MIN_HALF_LIFE = 2            # min half-life
# MAX_HALF_LIFE = 80          # max half-life

# for a, b, corr_val in selected:
#     try:
#         # Use only in-sample period for selection
#         df = selection_prices[[a, b]].dropna()
#         y = df[a]
#         x = df[b]

#         # Regression for beta
#         X = sm.add_constant(x)
#         model = sm.OLS(y, X).fit()
#         beta = model.params.iloc[1]  # fixed

#         # Spread & stationarity
#         spread = y - beta * x
#         spread = spread.dropna()
#         pvalue = adfuller(spread)[1]

#         # Half-life calculation
#         spread_lag = spread.shift(1)
#         spread_ret = spread - spread_lag
#         spread_lag = spread_lag.dropna()
#         spread_ret = spread_ret.dropna()

#         X_hl = sm.add_constant(spread_lag)
#         model_hl = sm.OLS(spread_ret, X_hl).fit()
#         half_life = -np.log(2) / model_hl.params.iloc[1]  # fixed

#         # Final filter
#         if (pvalue < ADF_P_THRESHOLD) and (MIN_HALF_LIFE < half_life < MAX_HALF_LIFE):
#             cointegrated.append((a, b, beta, pvalue, corr_val, half_life))

#     except Exception:
#         continue

# # Create dataframe
# coin_df = pd.DataFrame(
#     cointegrated,
#     columns=["A", "B", "beta", "adf_pval", "corr", "half_life"]
# )

# print("✅ FINAL PAIRS FOUND (2019-2024):", len(coin_df))
# coin_df.sort_values("adf_pval").head(10)


In [32]:
# # ===============================
# # 5️⃣ Spread volatility, score, inverse-vol weights
# # ===============================
# vols = []
# scores = []

# for _, row in coin_df.iterrows():
#     a, b, beta = row["A"], row["B"], row["beta"]
#     df = selection_prices[[a, b]].dropna()
#     y = df[a]
#     x = df[b]

#     spread = y - beta * x
#     norm_spread = spread / y.mean()

#     spread_vol = norm_spread.rolling(60).std().mean()
#     vols.append(spread_vol)

#     score = spread_vol / row["half_life"]
#     scores.append(score)

# coin_df["spread_vol"] = vols
# coin_df["score"] = scores
# coin_df["inv_vol"] = 1 / coin_df["spread_vol"]
# coin_df["weight"] = coin_df["inv_vol"] / coin_df["inv_vol"].sum()

# # Select top-K pairs
# top_k = 15
# coin_df = coin_df.sort_values("score", ascending=False).reset_index(drop=True)
# selected = coin_df.head(top_k).copy()

# # ===============================
# # 5️⃣b Ensure both tickers exist in 2025
# # ===============================
# valid_pairs = []
# for _, row in selected.iterrows():
#     if row["A"] in backtest_prices.columns and row["B"] in backtest_prices.columns:
#         valid_pairs.append(row)
# selected = pd.DataFrame(valid_pairs)

# print("Pairs available for 2025 OOS backtest:", len(selected))
# used = set()
# filtered = []

# for _, row in selected.iterrows():
#     if row["A"] not in used and row["B"] not in used:
#         filtered.append(row)
#         used.update([row["A"], row["B"]])

# selected = pd.DataFrame(filtered)



In [33]:
# # ===============================
# # 6️⃣ Backtest function
# # ===============================
# def backtest_pair(backtest_prices, A, B, beta, window=60, entry_z=2, exit_z=0.5, tc=0.001):
#     s = np.log(backtest_prices[A]) - beta * np.log(backtest_prices[B])
#     norm_s = s / backtest_prices[A].mean()

#     mean = norm_s.rolling(window).mean()
#     std = norm_s.rolling(window).std()
#     z = (norm_s - mean) / std
#     z = z.dropna()

#     if z.empty:
#         return None

#     pnl = []
#     pos_log = []
#     last_pos = 0

#     for i in range(1, len(z)):
#         zi = z.iloc[i]
#         date = z.index[i]
#         if zi > entry_z and last_pos == 0:
#             last_pos = -1
#             cost = tc * (abs(backtest_prices[A].loc[date]) + abs(backtest_prices[B].loc[date]))
#         elif zi < -entry_z and last_pos == 0:
#             last_pos = 1
#             cost = tc * (abs(backtest_prices[A].loc[date]) + abs(backtest_prices[B].loc[date]))
#         elif abs(zi) < exit_z and last_pos != 0:
#             last_pos = 0
#             cost = tc * (abs(backtest_prices[A].loc[date]) + abs(backtest_prices[B].loc[date]))
#         else:
#             cost = 0
#         # Skip low-signal days completely
#         if abs(zi) < 0.75:
#             spread_change = s.iloc[i] - s.iloc[i-1]
#             daily_pnl = last_pos * spread_change - cost
#             pnl.append(daily_pnl)


#         spread_change = s.iloc[i] - s.iloc[i-1]
#         daily_pnl = last_pos * spread_change - cost

#         pnl.append(daily_pnl)
#         pos_log.append(last_pos)

#     pnl = pd.Series(pnl, index=z.index[1:])
#     equity = pnl.cumsum()

#     sharpe = np.sqrt(252) * pnl.mean() / pnl.std() if pnl.std() > 0 else 0
#     maxdd = (equity - equity.cummax()).min()
#     trades = sum(1 for x in pos_log if x != 0)

#     return {"equity": equity, "pnl": pnl, "sharpe": sharpe, "maxdd": maxdd, "trades": trades}


In [34]:
def backtest_pair(backtest_prices, A, B, beta,
                  window=60, entry_z=1.6, exit_z=0.5, tc=0.001):
    """
    Backtest a single pair with realistic execution:
    - Lagged signal (t -> t+1)
    - Spread normalization
    - Transaction cost proportional to beta
    """
    
    # Log-price spread
    s = np.log(backtest_prices[A]) - beta * np.log(backtest_prices[B])
    
    # Rolling mean & std
    mean = s.rolling(window).mean()
    std  = s.rolling(window).std()
    
    # Lagged z-score for next-day execution
    z = ((s - mean) / std).shift(1)
    z = z.dropna()
    
    if z.empty:
        return None
    
    pnl = []
    pos_log = []
    last_pos = 0
    
    for i in range(1, len(z)):
        zi = z.iloc[i]
        date = z.index[i]
        
        # Momentum / mean-reversion logic
        if zi > entry_z and last_pos == 0:
            last_pos = 1
            cost = tc * (1 + abs(beta))
        elif zi < -entry_z and last_pos == 0:
            last_pos = -1
            cost = tc * (1 + abs(beta))
        elif abs(zi) < exit_z and last_pos != 0:
            last_pos = 0
            cost = tc * (1 + abs(beta))
        else:
            cost = 0
        
        # Spread change normalized by rolling std
        spread_change = s.iloc[i] - s.iloc[i-1]
        daily_pnl = last_pos * spread_change / std.iloc[i-1] - cost
        
        pnl.append(daily_pnl)
        pos_log.append(last_pos)
    
    pnl = pd.Series(pnl, index=z.index[1:])
    equity = pnl.cumsum()
    
    sharpe = np.sqrt(252) * pnl.mean() / pnl.std() if pnl.std() > 0 else 0
    maxdd = (equity - equity.cummax()).min()
    trades = sum(np.diff([0] + pos_log) != 0) // 2
    
    return {
        "equity": equity,
        "pnl": pnl,
        "sharpe": sharpe,
        "maxdd": maxdd,
        "trades": trades,
        "z": z.reindex(pnl.index)
    }


In [35]:
# ===============================
# 8️⃣ FINAL REALISTIC OOS BACKTEST
# ===============================

MIN_HOLD_DAYS = 5
TOP_K = 15
MAX_LEVERAGE = 1.25
TARGET_VOL = 0.008

results = []
pair_pnls = []
pair_zscores = []
pair_clusters = {}

# stock → cluster map
stock_cluster_map = dict(zip(tickers, labels))

# ===============================
# Per-pair backtests (signal only)
# ===============================
for _, row in selected.iterrows():
    A, B = row["A"], row["B"]
    similarity = row["corr"]

    cluster_id = stock_cluster_map.get(A)
    if cluster_id is None:
        continue

    # IS beta
    df = selection_prices[[A, B]].dropna()
    X = sm.add_constant(df[B])
    beta = sm.OLS(df[A], X).fit().params.iloc[1]

    if np.isnan(beta) or abs(beta) > 5:
        continue

    res = backtest_pair(
        backtest_prices, A, B, beta,
        window=60, entry_z=1.8, exit_z=0.75, tc=0.001
    )
    if res is None:
        continue

    pair_name = f"{A}-{B}"

    results.append({
        "A": A,
        "B": B,
        "corr": similarity,
        "cluster": cluster_id,
        "sharpe": res["sharpe"],
        "trades": res["trades"]
    })

    pair_pnls.append(res["pnl"].rename(pair_name))
    pair_zscores.append(res["z"].rename(pair_name))
    pair_clusters[pair_name] = cluster_id


# ===============================
# Align data
# ===============================
pair_pnls = pd.concat(pair_pnls, axis=1).fillna(0)
pair_zscores = pd.concat(pair_zscores, axis=1).fillna(0)

# ===============================
# T+1 EXECUTION DELAY
# ===============================
pair_pnls = pair_pnls.shift(1)
pair_zscores = pair_zscores.shift(1)

# ===============================
# DAILY TOP-15 + CLUSTER CAP
# ===============================
final_mask = pd.DataFrame(False, index=pair_pnls.index, columns=pair_pnls.columns)

for date in pair_pnls.index:
    z_today = pair_zscores.loc[date].abs().dropna()
    if z_today.empty:
        continue

    ranked = z_today.sort_values(ascending=False)

    used_clusters = set()
    selected_pairs = []

    for pair in ranked.index:
        cid = pair_clusters[pair]
        if cid in used_clusters:
            continue

        selected_pairs.append(pair)
        used_clusters.add(cid)

        if len(selected_pairs) == TOP_K:
            break

    final_mask.loc[date, selected_pairs] = True

pair_pnls = pair_pnls.where(final_mask, 0)

# ===============================
# MINIMUM HOLDING PERIOD
# ===============================
positions = pair_pnls.copy()
for col in positions.columns:
    active = positions[col] != 0
    hold = 0
    for i in range(len(active)):
        if active.iloc[i]:
            hold += 1
            if hold < MIN_HOLD_DAYS:
                positions.iloc[i, positions.columns.get_loc(col)] = 0
        else:
            hold = 0

pair_pnls = positions

# ===============================
# PORTFOLIO AGGREGATION
# ===============================
portfolio_pnl_raw = pair_pnls.sum(axis=1)

# ===============================
# VOL TARGETING (HARD CAPPED)
# ===============================
realized_vol = portfolio_pnl_raw.rolling(30, min_periods=10).std()
scaling = TARGET_VOL / realized_vol
scaling = (
    scaling.replace([np.inf, -np.inf], np.nan)
           .ffill()
           .fillna(0)
           .clip(0, MAX_LEVERAGE)
)

portfolio_pnl = portfolio_pnl_raw * scaling
portfolio_equity = portfolio_pnl.cumsum()

# ===============================
# METRICS
# ===============================
portfolio_sharpe = (
    np.sqrt(252) * portfolio_pnl.mean() / portfolio_pnl.std()
    if portfolio_pnl.std() > 0 else 0
)
portfolio_maxdd = (portfolio_equity - portfolio_equity.cummax()).min()

# ===============================
# REPORT
# ===============================
print("\n=== FINAL REALISTIC PORTFOLIO (2025 OOS) ===")
print(f"Portfolio Sharpe: {portfolio_sharpe:.3f}")
print(f"Portfolio Max Drawdown: {portfolio_maxdd:.2f}")
print(f"Pairs traded per day (max): {TOP_K}")
print(f"Candidate universe size: {len(results)}")
print(f"Unique clusters used: {len(set(pair_clusters.values()))}")
print(f"Non-zero PnL days: {(portfolio_pnl != 0).sum()} / {len(portfolio_pnl)}")



=== FINAL REALISTIC PORTFOLIO (2025 OOS) ===
Portfolio Sharpe: 1.560
Portfolio Max Drawdown: -0.08
Pairs traded per day (max): 15
Candidate universe size: 120
Unique clusters used: 3
Non-zero PnL days: 59 / 187


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# ===============================
# SAFETY CHECKS
# ===============================
required_vars = [
    "portfolio_pnl",
    "portfolio_equity",
    "active_pairs_per_day"
]

for v in required_vars:
    if v not in globals():
        raise ValueError(f"Missing required variable: {v}")

# ===============================
# METRICS
# ===============================
portfolio_sharpe = (
    np.sqrt(252) * portfolio_pnl.mean() / portfolio_pnl.std()
    if portfolio_pnl.std() > 0 else 0
)

drawdown = portfolio_equity - portfolio_equity.cummax()
portfolio_maxdd = drawdown.min()

print("\n=== PERFORMANCE SUMMARY ===")
print(f"Annualized Sharpe Ratio: {portfolio_sharpe:.3f}")
print(f"Max Drawdown: {portfolio_maxdd:.3f}")
print(f"Avg active pairs/day: {active_pairs_per_day.mean():.2f}")
print(f"Max active pairs/day: {active_pairs_per_day.max()}")

# ===============================
# 1️⃣ Equity Curve
# ===============================
plt.figure(figsize=(11, 4))
portfolio_equity.plot()
plt.title("Portfolio Equity Curve (OOS)")
plt.xlabel("Date")
plt.ylabel("Cumulative PnL")
plt.tight_layout()
plt.show()

# ===============================
# 2️⃣ Drawdown Curve
# ===============================
plt.figure(figsize=(11, 3))
drawdown.plot(color="red")
plt.title("Portfolio Drawdown")
plt.xlabel("Date")
plt.ylabel("Drawdown")
plt.tight_layout()
plt.show()

# ===============================
# 3️⃣ Daily PnL Distribution
# ===============================
plt.figure(figsize=(8, 4))
portfolio_pnl.hist(bins=50)
plt.title("Daily Portfolio PnL Distribution")
plt.xlabel("Daily PnL")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()



ValueError: Missing required variable: active_pairs_per_day

In [ ]:
%run performance_plots.py


Exception: File `'performance_plots.py'` not found.